Finance API homework

In [ ]:
import json
import pandas as pd
import json
import requests

Task 1: Fetch Basic Stock Data
- Prompt the user to input two or more stock symbols.
- Use the API to fetch data for the given symbols.
- Extract and display only the Stock Ticker, Company Name, and Current Market Price.

In [ ]:
apikey = ""
url1 = "https://yfapi.net/v6/finance/quote"
stocks1 = input("Choose at least two stocks, separated by a comma: ")

querystring = {"symbols": stocks1}
headers = {'x-api-key': apikey}
response1 = requests.request("GET", url1, headers = headers, params = querystring)
cleanResponse1 = response1.json()
df1 = pd.DataFrame(cleanResponse1['quoteResponse']['result'])
df1['regularMarketPrice'] = df1['regularMarketPrice'].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
print(df1[['symbol', 'shortName', 'regularMarketPrice']])

Task 2.1: Fetch Additional Data Using Modules
- Allow the user to choose a module from the Quote Summary Endpoint.

-Find:
-52 Week High
-52 Week Low
-Return on Assets (ROA)

- Make an API request using the chosen module.
- Convert the response into a Pandas DataFrame.
- Display the DataFrame.

In [ ]:
apikey = ""
url2_1 = "https://yfapi.net/v11/finance/quoteSummary/{symbol}"
module2_1 = input("Choose either financialData or summaryDetail module: ")
stock2_1 = input("Choose a stock: ")

querystring = {"modules": module2_1, "symbol": stock2_1}
headers = {'x-api-key': apikey}

response2_1 = requests.request("GET", url2_1.format(symbol = stock2_1), headers = headers, params = querystring).json()

if module2_1 == "summaryDetail":
 data2_1 = {"52 Week High": response2_1["quoteSummary"]["result"][0]["summaryDetail"]["fiftyTwoWeekHigh"]["raw"],
        "52 Week Low": response2_1["quoteSummary"]["result"][0]["summaryDetail"]["fiftyTwoWeekLow"]["raw"]}

if module2_1 == "financialData":
 data2_1 = {"Return on Assets (ROA)": response2_1["quoteSummary"]["result"][0]["financialData"]["returnOnAssets"]["raw"]}

df2_1 = pd.DataFrame([data2_1])
print(df2_1)

Task 2.2: Fetch Additional Data Using Modules

*  Find Current Trending Stocks , Their Actual Name and Ticker and Current Price as well as their 52 high and low.

In [ ]:
apikey = ""

trendURL = "https://yfapi.net/v1/finance/trending/US"
headers = {'x-api-key': apikey}

trendResponse = requests.get(trendURL, headers = headers).json()
trendStocks = [item["symbol"] for item in trendResponse["finance"]["result"][0]["quotes"]]
trendList = ",".join(trendStocks)
trendDF = pd.DataFrame(trendStocks, columns = ["symbol"])

In [ ]:
infoList = []

for stock in trendStocks:
    infoURL = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": stock}
    headers = {'x-api-key': apikey}

    infoResponse = requests.request("GET", infoURL, headers=headers, params=querystring).json()
    eachStock = {"symbol": stock,
            "longName": infoResponse["quoteResponse"]["result"][0]["longName"],
            "regularMarketPrice": infoResponse["quoteResponse"]["result"][0]["regularMarketPrice"]}
    infoList.append(eachStock)

infoDF = pd.DataFrame(infoList)
print(infoDF)

In [ ]:
yearList = []

for stock in trendStocks:
  yearURL = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
  querystring = {"modules": "summaryDetail"}
  headers = {"x-api-key": apikey}

  yearResponse = requests.get(yearURL, headers = headers, params = querystring).json()
  eachStock = {"symbol": stock,
    "52high": yearResponse["quoteSummary"]["result"][0]["summaryDetail"]["fiftyTwoWeekHigh"]["raw"],
    "52low": yearResponse["quoteSummary"]["result"][0]["summaryDetail"]["fiftyTwoWeekLow"]["raw"]}
  yearList.append(eachStock)

yearDF = pd.DataFrame(yearList)
print(yearDF)

In [ ]:
mergedDF = infoDF.merge(yearDF, on = "symbol", how = "left")
print(mergedDF)

mergedDF["regularMarketPrice"] = pd.to_numeric(mergedDF["regularMarketPrice"], errors='coerce')
mergedDF["regularMarketPrice"] = mergedDF["regularMarketPrice"].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
mergedDF["52high"] = pd.to_numeric(mergedDF["52high"], errors='coerce')
mergedDF["52high"] = mergedDF["52high"].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")
mergedDF["52low"] = pd.to_numeric(mergedDF["52low"], errors='coerce')
mergedDF["52low"] = mergedDF["52low"].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")